# Hybrid Virtual Sensors

In this tutorial we design a hybrid approach using the linear transfer function in frequency domain in combination with autoregressive Neural Networks. This is the fifth tutorial on the SoftSensor Toolbox building upon the other four. Please check out the previous tutorial for more detailed informations concerning the specific models as well as the hyperparameter optimization. Furthermore we utilise advanced techniques for stabelization in the predicition. 

Hybrid models are extremely useful for systems that have a *strong linear component with slight non-linear deviations*. For these cases, hybrid models can offer a strong benefit in terms of accuracy and computational effort compared to pure deep learning or classical models.

## Data Loading
The data loading process works similarly to the previous tutorials. A steering system under road conditions is used as the data basis. The linear antecedent of the system is significant, which is why the use of hybrid approaches is appropriate.

These are matlab files which require the use of a special function to read the data. The internal function `read_vehicle_data` provides the functionality to read the matfile for the individual sensors.

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
data_path = os.path.join(os.path.abspath(""), 'data')

input_sensors = [f'input_{i}' for i in range(1, 10)]
output_sensors = [f'output_{i}' for i in range(1, 4)]

file_names = [f for f in os.listdir(data_path) if
                    os.path.isfile(os.path.join(data_path, f))]
df_list = [pd.read_parquet(os.path.join(data_path, n)) for n in file_names]

## Data Preprocessing

To enable hybrid models, preprocessing is carried out in two steps. In the first step, the data is read into a `Meas_handling` class, filtered and scaled as explained in the previous tutorials.

In [ ]:
from softsensor.meas_handling import Meas_handling
fs = 1/np.mean(df_list[0].index.diff().dropna())
data_handle = Meas_handling(df_list[:2], file_names[:2], input_sensors, output_sensors, fs, df_list[2:], file_names[2:])

freq_lim = [12, 1250]
data_handle.Resample(fs=4096)
data_handle.Filter(freq_lim)
data_handle.Scale()

In the second step, the linear solution of the system is calculated. If another solution is known, for example from simulations, this can of course also be used. To calculate the linear solution, the Frequency Response Function (FRF) with the class `tf` is used. A detailed introduction to the calculation of linear solutions is described in `tutorial/01_linear_models.ipynb`. 

Then the difference between the linear solution and the original measurement data must be calculated. The use of the difference is necessary because the subsequent training with stability parameters leads to a simpler training with improved properties. 

As a last step, the `Meas_handling` class must be adapted so that the linear solution is also used as input in addition to the actual input sensors.

In [ ]:
from softsensor.linear_methods import tf
from softsensor.eval_tools import comp_batch
import numpy as np

# compute linear solution
tf_class = tf(window_size=1024, hop=512, fs=data_handle.fs)
tf_class.fit(data_handle.train_df, input_sensors, output_sensors)
_ = comp_batch([tf_class], data_handle, data_handle.train_names + data_handle.test_names, ['lin'])

# compute difference between linear solution and measured output
lin_labels = [f'{s}_lin' for s in output_sensors]
for df in data_handle.train_df + data_handle.test_df:
    for s, l in zip(output_sensors, lin_labels):
        df[f'{s}_diff'] = np.array(df[s]) - np.array(df[l])

# redefine input sensors for subsequent models
data_handle.input_sensors = input_sensors + lin_labels
data_handle.output_sensors = [f'{s}_diff' for s in output_sensors]

In [ ]:
data_handle.train_df[0]

## Define Model
We define an Autoregressive Neural Network (ARNN) to approximate the functional dependency between input and output as a simple Neural Network. The model is autoregressive as it feeds back in the past outputs into the equation, leading to:

$y_i(t+1) = f(\mathbf{x}_i(t, ..., t-w_x), \mathbf{y}_i(t, ..., t-w_y))$

Formally this model is called NARX (nonlinear autoregressive model with exogenious input) and takes:

 * number of input channels (`input_channels`)
 * number of output channels (`pred_size`)
 * window size (`window_size`)
 * recurretn window size (`rnn_window`)
 * neurons in the hidden layers (`hidden_size`)
 
as input. We need to define an extendet input space as `len(input_sensors + output_sensors)` to include the additional input from the linear solution.

In [ ]:
from softsensor.autoreg_models import ARNN

ARNN = ARNN(input_channels=len(input_sensors + output_sensors), pred_size=len(output_sensors), window_size=50, rnn_window=50,
            hidden_size=[128, 64, 32, 16], activation='leaky_relu')

## Model training

After we have defined the model, the next step is to adapt the model to the data. The training and validation data can be extracted from the predefined `data_handle` class and then trained using the `train_model` function. The setup for the training works similar to normal non-hybrid methods

In [ ]:
import torch.nn as nn
import torch.optim as optim

max_epochs = 30 # Max Epochs to train the Model
lr = 1e-4 # np.logspace(np.log10(1e-5), np.log10(1e-1), max_epochs, endpoint=True)
rel_perm = 5e-4 # gaussian noise added to signals
criterion = nn.MSELoss()
optimizer = optim.Adam(ARNN.parameters(), lr=lr)

train_loader, val_loader = data_handle.give_torch_loader(window_size=50 , keyword='training', batch_size=256,
                                                         rnn_window=50)

The training process is done as in `tutorials/02_ARNN.ipynb`. However, we introduce a change and use a scheduler for the stability parameter. Since we know that a negative stability score (SC) means stable behaviour, we use a high parameter if SC is greater than zero and a low parameter if SC is less than zero. 
We therefore check the SC inbetween the training process and adjust the stabelizer during training. The scheduling results in a SC close to zero after the training process, thus favouring stable behaviour without interfering too much with the main criterion.

We therefore use a log-function for a SC < 0 and a linear function for SC > 0 

In [ ]:
from softsensor.stab_scheduler import log_lin_stab
stab = log_lin_stab(ARNN, s0=1e-7, s1=1e-2, m=.001)

We visualize the Stability Parameter to show the specific advantages over simpler methods without scheduling

In [ ]:
import matplotlib.pyplot as plt
stab.track = False
sc_original = np.linspace(-1/np.sqrt(50), 5, 5000)
eta_original = [stab.get_stab(ARNN, sc=sco) for sco in sc_original]

fig, ax = plt.subplots(1,2, figsize=(8,4.5))
ax[0].plot(sc_original, eta_original, color='grey', alpha=.5)
ax[0].set_xlim(-.15, .1)
ax[0].set_ylim(-.001, .03)
ax[1].plot(sc_original, eta_original, color='grey', alpha=.5)
ax[1].set_ylim(-.001, .03)
ax[0].set_ylabel('stabelizer')
ax[0].set_xlabel('SC')
ax[1].set_xlabel('SC')

Afterwards we train our model similar to the previous tutorials

In [ ]:
from softsensor.train_model import train_model
results = train_model(ARNN, train_loader, max_epochs, optimizer,
                        device='cpu', criterion=nn.MSELoss(), val_loader=val_loader,
                        patience=5, print_results=True, stabelizer=stab,
                        give_results=True, rel_perm=rel_perm)

## Model Evaluation
To Evaluate our model on testing data we use the predefined functions `comp_pred`, which computes the prediction for a defined track in the data_handle class. As track we choose our testing track, which can be accessed using the internal variable `test_names`. Furthermore we compute

In [ ]:
from softsensor.eval_tools import comp_batch
train_pred = comp_batch([ARNN], data_handle, data_handle.train_names, ['ARNN'])
test_pred = comp_batch([ARNN], data_handle, data_handle.test_names, ['ARNN'])

Since we have only used one model for the hybrid modelling to calculate the difference between the linear and the measured solution, the linear solution must still be added in post-processing for the model and the output

In [ ]:
for i, df in enumerate(train_pred):
    for out_s in output_sensors:
        train_pred[i][f'{out_s}_ARNN'] = df[f'{out_s}_diff_ARNN'] + df[f'{out_s}_lin']

for i, df in enumerate(test_pred):
    for out_s in output_sensors:
        test_pred[i][f'{out_s}_ARNN'] = df[f'{out_s}_diff_ARNN'] + df[f'{out_s}_lin']

Afterwards we visualize the results

In [ ]:
cols = ["output_1", "output_1_lin", "output_1_ARNN"]
test_pred[0][cols].plot(figsize=(12,4))
test_pred[0][cols][3:3.02].plot(figsize=(12,4))

Lastly we compute a comparison for as a Mean-Squared-Error in Frequency domain (MSLE) as well as Mean Square Error(MSE)  in time domain.

In [ ]:
from softsensor.eval_tools import comp_error

error = comp_error(test_pred[0], output_sensors, fs, names=['ARNN', 'lin'], metrics=['MSE', 'MAE', 'JSD'], freq_range=freq_lim)
for n in ['MSE', 'MAE', 'JSD']:
    error.filter(regex=n, axis=0).plot.bar(ylabel=f'{n}', rot=45)